In [1]:
import pandas as pd
import numpy as np
import pickle

## Table Transactions

In [3]:
transactions = pd.read_pickle("pickles/transactions.pkl")

In [3]:
# Transformation des doublons en champ "quantité"
transactions = transactions.groupby(
    [
        "customer_id",
        "article_id",
        "t_dat",
        "price",
        "sales_channel_id"],
    as_index=False).agg(
    quantity=(
        "article_id",
        "count"))

In [1]:
# Transformation des dates
transactions['day_of_week'] = transactions['t_dat'].dt.day_of_week
transactions['day_of_year'] = transactions['t_dat'].dt.day_of_year

NameError: name 'transactions' is not defined

In [ ]:
transactions = transactions.sort_values('t_dat')

In [4]:
# Ajout de la notion de n° de semaine
last_day = transactions['t_dat'].max()
transactions['week_number'] = (last_day - transactions['t_dat']).dt.days // 7

In [5]:
transactions.to_pickle('pickles/transactions.pkl')
transactions.head()

,customer_id,article_id,t_dat,price,sales_channel_id,quantity,day_of_week,day_of_year,week_number
20523780,b6227fbf6968093cde31b5335a522c36827ed7cc715d01...,0685687004,2018-09-20,0.016542,2,1,3,263,104
1594079,0e129e8643d295d3f00e0d57f55d7d8dff13b82592f590...,0700828002,2018-09-20,0.050831,1,1,3,263,104
17533528,9b871e8d5636a7bad8c826012c7a2b1a04bc93d0075c6e...,0685683001,2018-09-20,0.025407,1,1,3,263,104
4498464,27c273bffa5d3b2558627f2bb5d8de121e2530cd7e2f10...,0541519024,2018-09-20,0.016932,1,1,3,263,104
23254377,ce698d3c10ec286f4aa65cee781ed8a9926eeb3fae330a...,0565379023,2018-09-20,0.008458,1,1,3,263,104


## Table Cart

In [7]:
carts = transactions.groupby(["customer_id", "t_dat", "sales_channel_id"], as_index = False).agg(
    total = ("price", "sum"),
    unique_articles = ("article_id", "count"),
    total_articles = ("quantity", "sum"),
    content = ("article_id", lambda x: list(x))
)


In [8]:
carts = carts.sort_values(['customer_id', 't_dat'])

In [8]:
carts['day_of_week'] = pd.to_datetime(carts['t_dat']).dt.day_of_week
carts['day_of_year'] = pd.to_datetime(carts['t_dat']).dt.day_of_year

### Ajout de l'intervalle entre deux paniers

In [10]:
carts['day_from_origin'] = (carts['t_dat'] - carts['t_dat'].min()).dt.days

In [11]:
carts['days_from_previous'] = carts['day_from_origin'].diff()

In [12]:
carts['previous_customer_id']=""
carts['previous_customer_id'].iloc[1:] = carts['customer_id'].iloc[0:-1]
carts.loc[carts['customer_id'] != carts['previous_customer_id'], 'days_from_previous'] = 0

/home/treb/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
carts['days_from_previous'] = carts['days_from_previous'].fillna(0).astype(int)
carts = carts.drop('previous_customer_id', 1)

/tmp/ipykernel_4570/2697235294.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  carts = carts.drop('previous_customer_id', 1)


In [17]:
carts.to_pickle('pickles/carts.pkl')
carts.head()

,customer_id,t_dat,sales_channel_id,total,unique_articles,total_articles,content,day_from_origin,days_from_previous
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,1,0.110119,3,3,"[0625548001, 0627759010, 0176209023]",98,0
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,2,0.010153,1,1,[0697138006],224,126
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2,0.050831,1,2,[0568601006],247,23
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,1,0.033864,2,2,"[0745232001, 0607642008]",308,61
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,2,0.044051,1,1,[0656719005],363,55


## Table client

In [2]:
customers = pd.read_pickle('pickles/customers.pkl')
carts = pd.read_pickle('pickles/carts.pkl')

In [3]:
carts_customers = carts.groupby('customer_id', as_index = False).agg(
    average_cart_articles = ("unique_articles", "mean"),
    average_cart_price = ("total", "mean"),
    total_carts = ('customer_id', 'count'),
    total_articles = ("total_articles", 'sum'),
    total_price = ("total", 'sum'),
)
carts_customers.head()

,customer_id,average_cart_articles,average_cart_price,total_carts,total_articles,total_price
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,1.900000,0.054393,10,21,0.543932
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.250000,0.100510,24,86,2.412237
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.142857,0.086646,7,18,0.606525
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000,0.060983,1,2,0.060983
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667,0.078282,6,13,0.469695


In [4]:
# Intervalle moyen entre chaque panier
carts_intervals = carts[carts['days_from_previous'] != 0].groupby('customer_id', as_index = False).agg(
    average_cart_interval = ('days_from_previous', 'mean')
)
carts_intervals['average_cart_interval'] = carts_intervals['average_cart_interval'] / 7
carts_intervals.head()

,customer_id,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,17.285714
3,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,19.142857
4,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,33.428571


In [5]:
customers = customers.merge(carts_customers, on = 'customer_id', how = 'left')
customers = customers.merge(carts_intervals, on = 'customer_id', how = 'left')
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,average_cart_articles,average_cart_price,total_carts,total_articles,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.900000,0.054393,10.0,21.0,0.543932,9.809524
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,3.250000,0.100510,24.0,86.0,2.412237,4.259740
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,2.142857,0.086646,7.0,18.0,0.606525,17.285714
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,2.000000,0.060983,1.0,2.0,0.060983,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,2.166667,0.078282,6.0,13.0,0.469695,19.142857


In [7]:
customers.to_pickle('pickles/customers.pkl')

## Table Articles

In [6]:
articles = pd.read_pickle('pickles/articles.pkl')
transactions = pd.read_pickle('pickles/transactions.pkl')

In [25]:
transaction_articles = transactions.groupby('article_id', as_index = False).agg(
    total_purchases = ('quantity', 'sum'),
    average_quantity = ('quantity', 'mean'),
    average_price = ('price', 'mean')
)
transaction_articles.head()

,article_id,total_purchases,average_quantity,average_price
0,0108775015,10841,1.438752,0.008109
1,0108775044,7250,1.279562,0.008093
2,0108775051,215,1.214689,0.004982
3,0110065001,1044,1.063136,0.020055
4,0110065002,539,1.118257,0.018087


In [26]:
articles = articles.merge(transaction_articles, on = 'article_id', how = 'left')

### Ajout d'une colonne has_image (True / False)

In [29]:
# Chargement de la liste des fichiers images
# (Obtenu par la commande `find -name  *.jpg > images.csv` en gros) 

images = pd.read_csv('images.csv')
images['article_id'] = images['file_path'].apply(
    lambda x: 
        x.split('/')[2].split('.jpg')[0]
).astype(str)
images = images.sort_values('article_id')
images.head()

,file_path,article_id
73653,./010/0108775015.jpg,0108775015
73652,./010/0108775044.jpg,0108775044
73651,./010/0108775051.jpg,0108775051
16811,./011/0110065001.jpg,0110065001
16810,./011/0110065002.jpg,0110065002


In [30]:
# Fusion des deux tableaux en Left join
article_images = articles.merge(images, on ="article_id", how = 'left')

In [31]:
article_images['has_image'] = article_images['file_path'].isna().apply(lambda x: not x)
articles = article_images.drop("file_path", 1)
articles.head()

/tmp/ipykernel_4570/1019122983.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  articles = article_images.drop("file_path", 1)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,total_purchases,average_quantity,average_price,has_image
0,0108775015,0108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,10841.0,1.438752,0.008109,True
1,0108775044,0108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,7250.0,1.279562,0.008093,True
2,0108775051,0108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,215.0,1.214689,0.004982,True
3,0110065001,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",1044.0,1.063136,0.020055,True
4,0110065002,0110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",539.0,1.118257,0.018087,True


In [32]:
# Sauvegarde finale
articles.to_pickle('pickles/articles.pkl')

## Nettoyage des tables

In [33]:
customers = pd.read_pickle('pickles/customers.pkl')
articles = pd.read_pickle('pickles/articles.pkl')

### Articles

In [34]:


# Étape 3: Gestion des NaN
 ## fichier 'article' 
   ### Detection des lignes avec valeurs manquantes
articles[articles.isna().any(axis =1)]
   ### Proportion du nombre de lignes avec NaN 
print("La proportion de lignes du fichier article avec une valeur manquante est de:", '{:.1%}'.format((articles['detail_desc'].isnull().sum() / articles.shape[0])))
   ### Choix de suppression de ces lignes et création d'un nouveau df 
articles['detail_desc'] = articles['detail_desc'].fillna('')




La proportion de lignes du fichier article avec une valeur manquante est de: 0.4%


#### À valider - Suppression des 995 articles qui n'ont jamais été achetés

In [35]:
articles = articles.dropna(subset=['total_purchases'])

In [36]:
articles.to_pickle('pickles/articles_clean.pkl')

### Clients

In [8]:

 ## fichier 'customer'
   ### Colonne FN
     #### Inspection colonne  
customers['FN'].head(30)
     #### Remplacement des NaN par 0
customers['FN'] = customers['FN'].fillna(0)

   ### Colonne Active
     #### Inspection colonne 
customers['Active'].head(30)
     #### Remplacement des NaN par 0
customers['Active'] = customers['Active'].fillna(0)

   ### Colonne Club member status
     #### Inspection colonne 
customers['club_member_status'].head(30)
     #### Remplacement des NaN par NON ACTIVE
customers['club_member_status'] = customers['club_member_status'].fillna('NON ACTIVE')
customers[customers['club_member_status']=='NON ACTIVE'] 

   ### Colonne Fashion News Frequency
     #### Inspection colonne 
customers['fashion_news_frequency'].head(30)     
customers[customers['fashion_news_frequency'].isna()]  # Pour voir le lien entre cette colonne (avec modalités NaN) et les autres colonnes 
     #### Option 1: Remplacement NaN par None
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].fillna('NONE')
     #### Option 2: Suppression des lignes qui ont des valeurs manquantes dans age et fashions_news_freq
#customers[(customers['fashion_news_frequency'].isna()) &(customers['age'].isna())]  --> detecter les lignes qui ont val manquantes dans 2 colonnes
#customers = customers.dropna(axis = 0, how='all', subset=['fashion_news_frequency', 'age']) --> suppression des lignes
     #### Option 3 (suggestion de Jean) -> garder les lignes avec NaN car explication logique

   ### Colonne age
     #### Inspection de la colonne 
customers['age'].head(30)
customers[customers['age'].isna()]  # Pour voir le lien entre cette colonne (avec modalités NaN) et les autres colonnes
     #### Option 1: Remplacement NaN par None
#customers['age'] = customers['age'].fillna('NONE')
     #### Option 2: Remplacement par la moyenne
customers['age'] = customers['age'].fillna(customers['age'].mean())



In [10]:
customers.to_pickle('pickles/customers_clean.pkl')

## Jointure Transactions - Customers - Articles

In [3]:
transactions = pd.read_pickle("pickles/transactions")
customers = pd.read_pickle("pickles/customers")
articles = pd.read_pickle("pickles/articles")

transactions = transactions.sample(frac = 0.1)

In [4]:
transactions = transactions.merge(customers, on="customer_id", how="left")

In [4]:
transactions = transactions.merge(articles, on="article_id", how="left")

In [5]:
transactions.to_pickle("pickles/transactions_all_infos")

In [6]:
transactions.head()

,customer_id,article_id,t_dat,price,sales_channel_id,quantity,day_of_week,day_of_year,FN,Active,...,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,total_purchases,average_quantity,average_price,has_image
0,06164dfaa997b0d4c51e321d09825df738db72f40fef84...,678942001,2019-05-22,0.016932,2,1,2,142,1.0,1.0,...,Ladieswear,11.0,Womens Tailoring,1003.0,Knitwear,"Short-sleeved jumper in a soft, fine-knit visc...",11671.0,1.0,0.016313,True
1,3eeeec7264479f541c8a6036dfd0ba0722c127a20540d1...,867300002,2020-04-09,0.025407,2,1,3,100,0.0,0.0,...,Ladieswear,15.0,Womens Everyday Collection,1010.0,Blouses,Sleeveless top in softly draping satin. Double...,760.0,1.0,0.020976,True
2,4e1c50b8dc8d5080c4def498baf75e60dfe4152ce301c0...,399223001,2019-05-15,0.033881,1,1,2,135,1.0,1.0,...,Divided,57.0,Ladies Denim,1016.0,Trousers Denim,"Jeggings in washed, superstretch denim with a ...",19604.0,1.0,0.030976,True
3,f0bba28bce9d6b7cf2aaa9b6bbe67b9c5e5079a6ad2a78...,720125008,2019-09-10,0.033508,2,1,1,253,0.0,0.0,...,Sport,5.0,Ladies H&M Sport,1005.0,Jersey Fancy,Sports tights in fast-drying functional fabric...,1933.0,1.0,0.032000,True
4,34121391358e8057d31b10667e970a37579549540da8b1...,883492001,2020-03-01,0.016932,1,1,6,61,1.0,1.0,...,Divided,52.0,Divided Accessories,1019.0,Accessories,Belt in imitation leather with a metal buckle....,40.0,1.0,0.015773,True


## Jointure Carts - Customers

In [7]:
carts = pd.read_pickle("pickles/carts")
customers = pd.read_pickle("pickles/customers")

In [8]:
carts = carts.merge(customers, on="customer_id", how="left")

In [9]:
carts.to_pickle("pickles/carts_all_infos")

In [10]:
carts.head()

,customer_id,t_dat,sales_channel_id,total,unique_articles,total_articles_x,content,day_of_week,day_of_year,day_from_origin,...,club_member_status,fashion_news_frequency,age,postal_code,average_cart_articles,average_cart_price,total_carts,total_articles_y,total_price,average_cart_interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,1,0.110119,3,3,"[176209023, 625548001, 627759010]",3,361,98,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,2,0.010153,1,1,[697138006],3,122,224,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2,0.050831,1,1,[568601006],5,145,247,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,1,0.033864,2,2,"[607642008, 745232001]",3,206,308,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,2,0.044051,1,1,[656719005],2,261,363,...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,1.9,0.054393,10.0,19.0,0.543932,68.666667
